In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation


from quick_sim import setup_sim




# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, coupled_flux_qubit_non_linear_approx_pot_break_down
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation


coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

# parameters

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.1
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 1
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F
L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 


m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
_lambda, _theta, _eta

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1


In [ ]:
C_1

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]



# µ = 0.06, φ2xdc = 1.79, φ1x = 0.59, and φ2x = 0.10.
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

### protocol setting

In [ ]:
zeroDissipation = False
zeroDissipation = True
saveAllStates = True
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_1_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


# bookmark
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
phi_1_dcx_off, phi_2_dcx_off, M_12_off

m_12_factor = 1
CE_1_a = {
    "phi_1_x": phi_1_x_on_12 * 0.8, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
    "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}
CE_1_b = {
    "phi_1_x": phi_1_x_on_12  , "phi_2_x": phi_2_x_on_12, "M_12": M_12_on * m_12_factor, \
    "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}


# longer CE
t_duration = 100
protocol_list_LCE = [
    create_CE_Protocol(t_duration, CE_8),
    create_CE_Protocol(t_duration, CE_1),
    create_CE_Protocol(t_duration, CE_6),
    create_CE_Protocol(t_duration, CE_3),
    create_CE_Protocol(t_duration, CE_8),
    create_CE_Protocol(t_duration, four_well)
]
# shorter CE
protocol_list = [
    # forward
    create_CE_Protocol(50, CE_1_a),
    create_CE_Protocol(150, CE_1_b),
    create_CE_Protocol(150, CE_1_a),
    create_CE_Protocol(50, four_well)
    # create_CE_Protocol(100, CE_8),
    # create_CE_Protocol(100, flip_V),
    # create_CE_Protocol(100, flip_H),

    # create_CE_Protocol(100, CE_3),
    # create_CE_Protocol(100, CE_8)
]


protocol_list_variable_duration = [
    # forward
    create_CE_Protocol(30, CE_1_a),
    create_CE_Protocol(60, CE_1_b),
    create_CE_Protocol(30, CE_1_a),
    create_CE_Protocol(100, four_well)

]


protocol_list_fixed_duration = [
    # forward
    create_CE_Protocol(100, CE_1_b),
    # create_CE_Protocol(100, CE_1_b),
    # create_CE_Protocol(100, CE_1_a),
    create_CE_Protocol(100, four_well)

]


protocol_list = protocol_list_fixed_duration

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

In [ ]:
print(f"L: {L_factor}, C: {C_factor}, M_12: {M_12_on}")

### first initialization

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""

params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state

In [ ]:
cfqr.system.k_BT / k_B

In [ ]:
import numpy as np
from scipy import integrate
beta_1 = 2.3
gamma_1 = 10


[integrate.dblquad(f, -k * np.pi, k * np.pi, -k * np.pi, k* np.pi)[0] for k in range(1, 20)]

In [ ]:
def calculatePartitionFunction(params):
    gamma_1, gamma_2 = 10, 10
    f_1 = lambda y, x: np.exp(-(1/2 * x**2 + 1/2 * gamma_1 * y**2 + beta_1 * np.cos(x) + np.cos(y/2)))
    f_2 = lambda y, x: np.exp(-(1/2 * x**2 + 1/2 * gamma_2 * y**2 + beta_2 * np.cos(x) + np.cos(y/2)))    
    f_v1 = lambda x: np.exp(-1/2 * x**2)
    f_v2 = lambda x: np.exp(-1/2 * x**2)
    f_v3 = lambda x: np.exp(-1/2 * 1/4 * x**2)
    f_v4 = lambda x: np.exp(-1/2 * 1/4 * x**2)
    
    Z_1 = integrate.dblquad(f_1, -k * np.pi, k * np.pi, -k * np.pi, k* np.pi)
    Z_2 = integrate.dblquad(f_2, -k * np.pi, k * np.pi, -k * np.pi, k* np.pi)
    Z_v1 = integrate.quad(f_v1, -np.inf, np.inf)
    Z_v2 = integrate.quad(f_v2, -np.inf, np.inf)
    Z_v3 = integrate.quad(f_v3, -np.inf, np.inf)
    Z_v4 = integrate.quad(f_v4, -np.inf, np.inf)
    
    return Z_1[0] * Z_2[0] * Z_v1[0] * Z_v2[0] * Z_v3[0] * Z_v4[0]
    

In [ ]:
Z_tot = calculatePartitionFunction(10)

In [ ]:
U0_1

In [ ]:
NT = 10000
mins, maxes = -0.1, 0.1
v_c = x_c/t_c

test_coords = np.zeros((NT, cfqr.potential.N_dim, 2))
test_coords[:, 0, 0] = np.random.uniform(mins, maxes, NT)
test_coords[:, 1, 0] = np.random.uniform(mins, maxes, NT)
test_coords[:, 2, 0] = np.random.uniform(mins, maxes, NT)
test_coords[:, 3, 0] = np.random.uniform(mins, maxes, NT)


m_1, m_2, m_3, m_4 = C_1, C_2, C_1/4, C_2/4
k_BT = k_BT
test_coords[:, 0, 1] = np.random.normal(0, np.sqrt(k_BT/m_1), NT) / v_c
test_coords[:, 1, 1] = np.random.normal(0, np.sqrt(k_BT/m_2), NT) / v_c
test_coords[:, 2, 1] = np.random.normal(0, np.sqrt(k_BT/m_3), NT) / v_c
test_coords[:, 3, 1] = np.random.normal(0, np.sqrt(k_BT/m_4), NT) / v_c


In [ ]:
np.exp(-cfqr.system.get_energy(test_coords, 0)/U0_1)

In [ ]:
Z_tot

In [ ]:
U0_1

In [ ]:
_prob = np.exp(-cfqr.system.get_energy(test_coords, 0)/U0_1)/Z_tot
plt.hist(_prob)
plt.yscale("log")

In [ ]:
 decide = np.random.uniform(0, 1, NT)

In [ ]:
decide

In [ ]:
def eq_state(self, Nsample, t=None, resolution=500, beta=1, manual_domain=None, axes=None, slice_vals=None, verbose=True):
    '''
    function still in development, docstring will come later.
    generates Nsample coordinates from an equilibrium distribution at
    time t.

    '''
    if self.potential.N_dim >= 4 and (axes is None or len(axes) > 4):
        if resolution > 100:
            print('using a lower resolution for searching a space in >3 dimensions')

            if self.potential.N_dim >= 4:
                resolution = 50
            print('new resolution is {}'.format(resolution))

    NT = Nsample
    state = np.zeros((max(100, int(2*NT)), self.potential.N_dim, 2))

    def get_prob(self, state):
        Z_tot = 24.936095149247745
        

        
        
        return np.exp(-U) / Z_tot 

    if t is None:
        t = self.protocol.t_i

    U, X = self.lattice(t, resolution, axes=axes, slice_values=slice_vals, manual_domain=manual_domain)
    mins = []
    maxes = []
    for item in X:
        mins.append(np.min(item))
        maxes.append(np.max(item))

    U0 = np.min(U)
    i = 0

    count = 0
    while i < Nsample:
        count += 1
        test_coords = np.zeros((NT, self.potential.N_dim, 2))

        test_coords[:, 0, 0] = np.random.uniform(mins, maxes, NT)
        test_coords[:, 1, 0] = np.random.uniform(mins, maxes, NT)
        test_coords[:, 2, 0] = np.random.uniform(mins, maxes, NT)
        test_coords[:, 3, 0] = np.random.uniform(mins, maxes, NT)
        

        if self.has_velocity:
            m_1, m_2, m_3, m_4 = self.capacitance
            k_BT = self.k_BT
            test_coords[:, 0, 1] = np.random.normal(0, np.sqrt(k_BT/m_1), Nsample) / self.v_c
            test_coords[:, 1, 1] = np.random.normal(0, np.sqrt(k_BT/m_2), Nsample) / self.v_c
            test_coords[:, 2, 1] = np.random.normal(0, np.sqrt(k_BT/m_3), Nsample) / self.v_c
            test_coords[:, 3, 1] = np.random.normal(0, np.sqrt(k_BT/m_4), Nsample) / self.v_c
        
        p = get_prob(self, test_coords)

        decide = np.random.uniform(0, 1, NT)
        n_sucesses = np.sum(p > decide)
        if i == 0:
            ratio = max(n_sucesses/NT, .05)

        state[i:i+n_sucesses, :, :] = test_coords[p > decide, :, :]
        i = i + n_sucesses
        if verbose:
            print("\r found {} samples out of {}".format(i, Nsample), end="")

        NT = max(int((Nsample-i)/ratio), 100)

    print("\nfrom system: finish the while loop in sampling.")
    state = state[0:Nsample, :, :]
    # print("the state is", state)

    
    else:
        return state[:, :, 0]

    return state


# check before simulation

### parameter graph

In [ ]:
cfqr.showPamameterTimeGraph(protocol_time_array, protocol_all_time_array)

### potential shape check

In [ ]:
coupled_flux_qubit_non_linear_approx_pot(-1.96, 0, 0, 0, initial_parameter_dict.values())

In [ ]:
_t = 100

critical_points = np.array(cfqr.get_critical_points_at_t_for_a_set_of_parameter(_t, showPotential=False, verbose = False, guess=[(0, 0), (-2, 0), (2, 0), (0,2), (0, -2), (-2, -2), (-2, 2), (2, -2), (2, 2)]))




l = cfqr.plotGraph(t = _t, cutlineValue=critical_points[1][0], x_range=[-3, 3],y_range = [800, 4000], contour_range = [800, 5000], showGraph = True)
contour_plt = l["ax0"]
cutline_plt = l["ax1"]
cutline_plt.set_xlim(-5, 5)
cutline_plt.set_ylim(500, 1700)

contour_plt.scatter(critical_points[:, 0], critical_points[:, 1])
print([0])

# cfqr.plotGraph(t = 50, cutlineValue=-1.96, showGraph=True)
plt.show()

### bifurcation check

In [ ]:
_stepped_time = protocol_all_time_array[::100]
minimum_points_of_00 = [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = [(-2, -2)])[0] for _t in _stepped_time]
fig, ax = plt.subplots(1, 2, figsize = [8, 4])

ax[0].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 0])
ax[0].set_xlabel(r't $(\tau_c)$', fontsize = 12)
ax[0].set_ylabel(r'$\varphi_1$', fontsize = 12)

ax[1].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 1])
ax[1].set_xlabel(r't $(\tau_c)$', fontsize = 12)
ax[1].set_ylabel(r'$\varphi_2$', fontsize = 12)
fig.tight_layout()

# actual simulations

In [ ]:
importlib.reload(cfq_batch_sweep)
as_step = np.s_[::100]

# step 5: perform simulations
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = as_step, measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)

# sim Results

### fidelity

In [ ]:
plotFidelityBarChart(simResult['cfqr'].sim.fidelity_time_array)

In [ ]:
def cutlineDataGeneratingFunction(initial_parameter_dict, direction = 'v', color = 'red', guess = [(-2, -2)]):
    coord_index = 0 if direction == "v" else 1
    def tracking_minimum_point(_t):
        return  [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = guess)[0][coord_index], direction, color]
    return tracking_minimum_point

### animation

In [ ]:
importlib.reload(coupled_fq_protocol_library)

In [ ]:
pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}

graph_setting = {
    0.1: {"vmin": 1000, "vmax": 2000, "ymin": 1000, "ymax": 3000},
    1: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
    2: {"vmin": 50, "vmax": 250, "ymin": 50, "ymax": 250},    
    5: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
    20: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20}
}

targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline(simResult['cfqr'], time_array=targetTimeArray, frame_skip = 5, 
                                                                               vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K",
                                                                               params=params, cutlineGeneratingFunction = cutlineDataGeneratingFunction(initial_parameter_dict),
                                                                               pColor=pColor, cutlineXLimit=[-4, 4], cutlineYLimit=[graph_setting[T]['ymin'],graph_setting[T]['ymax']])

writer = animation.PillowWriter(fps=15,
                    metadata=dict(artist='Me'),
                    bitrate=1800)
ani.save('scatter.gif', writer=writer)
Image(filename="scatter.gif")

In [ ]:

video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

### barrier height analysis

In [ ]:
target_time_array = protocol_all_time_array[simResult['cfqr'].as_step]

In [ ]:
param_list_t_array = [simResult['cfqr'].protocol.get_params(_t) for _t in target_time_array]

In [ ]:
param_list_t_array[0]

In [ ]:
plt.plot(target_time_array, np.array(min_x_00)[:, 0])

In [ ]:
min_x_00_PE = [coupled_flux_qubit_non_linear_approx_pot(p[0], p[1], _list[10], _list[11], _list) for p, _list in zip(min_x_00, param_list_t_array)]
min_x_01_PE = [coupled_flux_qubit_non_linear_approx_pot(p[0], p[1], _list[10], _list[11], _list) for p, _list in zip(min_x_01, param_list_t_array)]
mid_PE =      [coupled_flux_qubit_non_linear_approx_pot(p[0], p[1], _list[10], _list[11], _list) for p, _list in zip(mid_point, param_list_t_array)]

In [ ]:
plt.plot(target_time_array, min_x_00_PE, label = "00")
plt.plot(target_time_array, min_x_01_PE, label = "01")
plt.plot(target_time_array, mid_PE, label = "01")
plt.legend()

In [ ]:
Image(filename="scatter.gif")

In [ ]:
min_x_00 = [simResult['cfqr'].get_minimum_at_for_a_set_of_parameter(_t, initial_parameter_dict, guess=[(-2, -2)])[0] for _t in protocol_all_time_array[simResult['cfqr'].as_step]]

In [ ]:
mid_point = np.array([(x[0], 0) for x in min_x_00])

In [ ]:
min_x_01 = np.array([(x[0], -x[1]) for x in min_x_00])

### KE analysis

In [ ]:
index_of_00 = simResult['cfqr'].getIndexOfParticles()["00"]
coord = simResult['cfqr'].sim.output.all_state['states'][:, 100, ...]
print(np.mean(simResult['cfqr'].system.get_kinetic_energy(coord)))


In [ ]:
plt.hist(simResult['cfqr'].system.get_kinetic_energy(coord))

In [ ]:
showAnimation = False
showAnimation = True
if showAnimation:
    pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
    simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip= 10, 
color_key = True, color_bar_range = [240, 600])


In [ ]:


showGif = False
showGif = True
if showGif:
    simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict, pColor = pColor, frame_skip= 5, showFormat= "gif")
Image(filename="scatter.gif")

In [ ]:
keyStep_work_distribution = simResult['cfqr'].sim.keyStep_work_distribution

In [ ]:
particle_index = cfqr.getIndexOfParticles()


In [ ]:
keyStep_work_distribution_00 = [_dist[particle_index['00']] for _dist in keyStep_work_distribution]
keyStep_work_distribution_01 = [_dist[particle_index['01']] for _dist in keyStep_work_distribution]
keyStep_work_distribution_10 = [_dist[particle_index['10']] for _dist in keyStep_work_distribution]
keyStep_work_distribution_11 = [_dist[particle_index['11']] for _dist in keyStep_work_distribution]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = [8, 8])
ax_flatten = ax.flatten()
for i, x in enumerate(['00', '01', '10', '11']):
    ax_flatten[i].hist(keyStep_work_distribution[-1][particle_index[x]])

In [ ]:
for x in keyStep_work_distribution_00[1:]:
    plt.hist(x, bins = 40)
    plt.show()

In [ ]:
for x in keyStep_work_distribution_01[1:]:
    plt.hist(x, bins = 40)
    plt.show()

In [ ]:
for x in keyStep_work_distribution_11[1:]:
    plt.hist(x, bins = 40)
    plt.show()

In [ ]:
for x in keyStep_work_distribution_11[1:]:
    plt.hist(x, bins = 40)
    plt.show()

In [ ]:
def plotState(state):
    plt.scatter(state[:,0,0], state[:,1,0])

In [ ]:

keyStepIndex = 2
keyStepState =  simResult['cfqr'].sim.keyStep_all_states[keyStepIndex]
keyStepWokrDistribution = simResult['cfqr'].sim.keyStep_work_distribution[keyStepIndex]
fig, ax = plt.subplots(1, 2, figsize = [9.5, 4])
# ax[0].hist(keyStepWokrDistribution, bins = 50)
ax[1].scatter(keyStepState[:,0,0], keyStepState[:,1,0])

class_1_filter = keyStepWokrDistribution < -10 
class_2_filter = np.logical_and(keyStepWokrDistribution > -10, keyStepWokrDistribution < 10 )
class_3_filter = keyStepWokrDistribution > 10 
class_1_mean = np.mean(keyStepWokrDistribution[class_1_filter])
class_2_mean = np.mean(keyStepWokrDistribution[class_2_filter])
class_3_mean = np.mean(keyStepWokrDistribution[class_3_filter])



# ax[0].hist(keyStepWokrDistribution, bins = 50, color="red")
class_color = ["red", "orange", "blue"]
bins=np.histogram(keyStepWokrDistribution, bins=50)[1] #get the bin edges

ax[0].hist(keyStepWokrDistribution[class_1_filter], bins = bins, color="red")
ax[0].hist(keyStepWokrDistribution[class_2_filter], bins = bins, color="orange")
ax[0].hist(keyStepWokrDistribution[class_3_filter], bins = bins, color="blue")

ax[1].scatter(keyStepState[class_1_filter][:,0,0], keyStepState[class_1_filter][:,1,0], color = 'red')
ax[1].scatter(keyStepState[class_2_filter][:,0,0], keyStepState[class_2_filter][:,1,0], color = 'orange')
ax[1].scatter(keyStepState[class_3_filter][:,0,0], keyStepState[class_3_filter][:,1,0], color = 'blue')

plt.show()


fig, ax = plt.subplots(1, 3, figsize = [12, 4])
ax[0].hist(keyStepWokrDistribution[class_1_filter], bins = 50, color="red")
ax[0].hist(keyStepWokrDistribution[class_2_filter], bins = 50, color="orange")
ax[0].hist(keyStepWokrDistribution[class_3_filter], bins = 50, color="blue")
ax[0].set_xlim(class_1_mean-2, class_1_mean+2)
ax[0].text(x=0.70, y=0.90, s=f"N = {np.sum(class_1_filter)}", fontsize=12, transform=ax[0].transAxes)


ax[1].hist(keyStepWokrDistribution[class_1_filter], color="red")
ax[1].hist(keyStepWokrDistribution[class_2_filter], color="orange")
ax[1].hist(keyStepWokrDistribution[class_3_filter], color="blue")
ax[1].set_xlim(class_2_mean - 2, class_2_mean + 2); ax[1].set_ylim(0, 80)
ax[1].text(x=0.70, y=0.90, s=f"N = {np.sum(class_2_filter)}", fontsize=12, transform=ax[1].transAxes)

ax[2].hist(keyStepWokrDistribution[class_1_filter], color="red")
ax[2].hist(keyStepWokrDistribution[class_2_filter], color="orange")
ax[2].hist(keyStepWokrDistribution[class_3_filter], color="blue")
ax[2].set_xlim(class_3_mean - 2, class_3_mean + 2); ax[2].set_ylim(0, 100)
ax[2].text(x=0.70, y=0.90, s=f"N = {np.sum(class_3_filter)}", fontsize=12, transform=ax[2].transAxes)
plt.show()

In [ ]:
# simResult['cfqr'].getFidelityTimeGraph()

### work analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
mean_work_stand_error = np.std(simResult["work_distribution"]) / np.sqrt(params['N'])
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))
print("sim_id = ", simResult['simulation_data'])
print(f"N = {params['N']}, dt = {params['dt']}")
# print(f"phi_1dcx_on: {phi_1_dcx_on}, phi_2dcx_on: {phi_2_dcx_on}, M_12_on: {M_12_on}")
print(f"L_factor = {L_factor}, C_factor = {C_factor}, I_m_factor = {I_m_factor}, gamma = {gamma}")
print([x["duration"] for x in protocol_list])
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work:.3g} ± {3 * mean_work_stand_error:.3g}')


In [ ]:
group_1_filter = simResult["work_distribution"] < 60
group_2_filter = simResult["work_distribution"] > 60

In [ ]:
plt.hist(simResult["work_distribution"][group_2_filter], bins = 40)

In [ ]:
plt.hist(simResult["work_distribution"][group_1_filter], bins = 40)

In [ ]:
# step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
step_array = simResult['cfqr'].sim.target_step_index
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

plt.errorbar(step_array[::skip_step], work_mean[::skip_step], yerr = work_std[::skip_step])
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', linestyle ='--', label = 'axvline - full height')
    # plt.annotate(f"{i+1}", xy=(t, 4.5),  color='red', ha='center', size=16)

print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:

target_time_index = np.array(protocol_time_array)/params['dt']
target_time_index = target_time_index.astype("int") - 1

# work_done_array = [work_mean[i] for i in target_time_index[1:]]
work_done_array = simResult["cfqr"].sim.keyStep_work_statistic[:,0]
work_done_of_each_step = np.array(work_done_array)[1:] - np.array(work_done_array)[:-1]

Measure_Erasue_steps = [0, 1, 2, 3]
Control_Steps = [4, 5, 6, 7]

print("Third way to split the steps: (E, 0-1), (M. 2), (C: 3-6)")
print(f"work_done = {work_done_of_each_step}" )
print("measurement + reset = ", np.sum([work_done_of_each_step[i] for i in Measure_Erasue_steps]))
print("control = ", np.sum([work_done_of_each_step[i] for i in Control_Steps]))
print(sum(work_done_of_each_step))

# check states

# animations

In [ ]:
simResult['cfqr'].showAnimation(protocol_list, computation_protocol_parameter_dict)

In [ ]:
from edward_tools.visualization import separate_by_state

In [ ]:
self = simResult['cfqr']
t = 100
resolution = 100

fig, ax = plt.subplots(figsize=(6,6))
protocol_key = ['U0_1', 'U0_2', 'gamma_1', 'gamma_2', 'beta_1', 'beta_2', 'd_beta_1', 'd_beta_2', 'phi_1_x', 'phi_2_x', 'phi_1_dcx', 'phi_2_dcx', 'M_12', 'x_c']
contour_plt = plt


phi_1_dcx_index = protocol_key.index('phi_1_dcx')
phi_2_dcx_index = protocol_key.index('phi_2_dcx')
phi_1_dc_i = self.protocol.get_params(t)[phi_1_dcx_index]
phi_2_dc_i = self.protocol.get_params(t)[phi_2_dcx_index]
slice_values = [0, 0, phi_1_dc_i, phi_2_dc_i]

modified_manual_domain = [(manual_domain[0][1], manual_domain[0][0]), \
                              (manual_domain[1][1], manual_domain[1][0])]
U, X_mesh = self.system.lattice(t, resolution, axes=(0, 1),\
                                    manual_domain=modified_manual_domain, slice_values = slice_values)

# 5. find vmin, vmax, x_min, x_max, y_min and y_max
vmin, vmax = np.min(U), np.max(U)
x_min, x_max = np.min(X), np.max(X)
y_min, y_max = np.min(Y), np.max(Y)

all_state = self.sim.output.all_state['states']
phi_1_dc = all_state[..., 2, 0]
phi_2_dc = all_state[..., 3, 0]
all_state = all_state[:, :, (0, 1), :]

N, nsteps, N_dim = np.shape(all_state)[0], np.shape(all_state)[1], np.shape(all_state)[2]
X, Y = X_mesh[0], X_mesh[1]
which_axes = [np.s_[..., i, 0] for i in range(N_dim)]
x_array = [all_state[item] for item in which_axes]
x = x_array[0]
y = x_array[1]
state_lookup = separate_by_state(all_state, has_velocity = True)
pColor = {"00": "#3FC7F2", "01": "#FCEF51", "10": "#BEDC8C", "11": "#F7B6D2"}
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F7B6D2"}

cont = ax.contourf(X, Y, U, 40, vmin = 0, vmax = 400, cmap = "afmhot")
scat = [ax.scatter(x[state_lookup[key], 120], y[state_lookup[key], 120], c = pColor[key]) for key in state_lookup]

# phi_1dc, phi_2dc and gamma

In [ ]:
general_analysis_tool = importlib.reload(general_analysis_tool)
general_analysis_tool.show_phi_dc_with_time(simResult['cfqr'], params)

# save result

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
params['comment'] = "test for save all_states"

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

# work_analysis

In [ ]:
selected_work_distribution = simResult["work_distribution"] < -0.2
# selected_work_distribution = simResult["work_distribution"] > 1.5
# selected_work_distribution = np.logical_and(simResult["work_distribution"] > -0.2, simResult["work_distribution"] < 1.5)

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 1000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
selected_particles = simResult["cfqr"].sim.output.all_state["states"][selected_work_distribution, ...]
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(selected_particles, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
step_time_array = np.array(simResult["cfqr"].sim.output.all_state["step_indices"]) * params['dt']
potential_array = []
for _index, _t in enumerate(step_time_array):
    _params_at_t = simResult['cfqr'].system.protocol.get_params(_t)
    def cfq_potential(state):
        # print(state)
        phi_1, phi_2, phi_1dc, phi_2dc = state
        return coupled_flux_qubit_non_linear_approx_pot(phi_1, phi_2, phi_1dc, phi_2dc, _params_at_t)
    vectorized_cfq_potential = np.vectorize(cfq_potential, signature='(4)->()')
    potential_array.append(vectorized_cfq_potential(selected_particles[:, _index, :, 0]))
    
    

In [ ]:
plt.plot(potential_array)
plt.show()

In [ ]:
protocol_time_array, protocol_time_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']

In [ ]:
minimum_value_data_points = []
importlib.reload(minimum_value_of_potential)
for t in protocol_all_time_array[::1000]:
    minimum_value_data = minimum_value_of_potential.find_minimum_for_all_potential_2(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False)
    minimum_value_data_points.append(minimum_value_data[1])

In [ ]:
minimum_value_of_potential = importlib.reload(minimum_value_of_potential)

In [ ]:
minimum_value_data_points = np.array(minimum_value_data_points)

In [ ]:
offset_value = minimum_value_data_points[1:] - minimum_value_data_points[:-1]

In [ ]:
all_minimum_value_at_t = []

for t in protocol_time_array:
    minimum_value_data = minimum_value_of_potential.find_all_minimum_points_for_all_potential(cfqr, t,  beta_1, beta_2, d_beta_1, d_beta_2, verbose = False, guess=[[-2, -2], [-2, 2], [2, -2], [2, 2]])
    all_minimum_value_at_t.append([minimum_value_data, t])

In [ ]:
all_minimum_value_at_t

In [ ]:
plt.plot(minimum_value_data_points)

In [ ]:
plt.plot(protocol_all_time_array[::10], minimum_value_data_points)

In [ ]:
import numpy as np
from scipy import integrate
beta_1 = 2.3
gamma_1 = 10
f = lambda y, x: np.exp(-(1/2 * x**2 + 1/2 * gamma_1 * y**2 + beta_1 * np.cos(x) + np.cos(y/2)))

[integrate.dblquad(f, -k * np.pi, k * np.pi, -k * np.pi, k* np.pi)[0] for k in range(1, 20)]

In [ ]:
f = lambda y: np.exp(-(1/2 * x**2 ))